In [ ]:
!pip install --trusted-host pypi.org --trusted-host files.pythonhosted.org tabulate
!pip install --trusted-host pypi.org --trusted-host files.pythonhosted.org seaborn
!pip install --trusted-host pypi.org --trusted-host files.pythonhosted.org imblearn

%matplotlib inline
import numpy as np
import pandas as pd
import pylab as pl
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn import preprocessing
from sklearn import datasets
from sklearn.metrics import confusion_matrix
from tabulate import tabulate
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [ ]:
# Función para visualizar un conjunto de datos en 2D
def plot_data(X, y, size=None):
    y_unique = np.unique(y)
    colors = pl.cm.rainbow(np.linspace(0.0, 1.0, y_unique.size))
    for this_y, color in zip(y_unique, colors):
        this_X = X[y == this_y]
        pl.scatter(this_X[:, 0], this_X[:, 1],  c=color,
                   alpha=0.5, edgecolor='k', s=size,
                    label="Class %s" % this_y)
    pl.legend(loc="best")
    pl.title("Data")
    
# Función para visualizar de la superficie de decisión de un clasificador
def plot_decision_region2(X, pred_fun):      #Función para visualizar la superficie de decisión de nuestro algoritmo.
    min_x = np.min(X[:, 0])
    max_x = np.max(X[:, 0])
    min_y = np.min(X[:, 1])
    max_y = np.max(X[:, 1])
    min_x = min_x - (max_x - min_x) * 0.05
    max_x = max_x + (max_x - min_x) * 0.05
    min_y = min_y - (max_y - min_y) * 0.05
    max_y = max_y + (max_y - min_y) * 0.05
    x_vals = np.linspace(min_x, max_x, 100)
    y_vals = np.linspace(min_y, max_y, 100)
    XX, YY = np.meshgrid(x_vals, y_vals)
    grid_r, grid_c = XX.shape
    ZZ = np.zeros((grid_r, grid_c))
    for i in range(grid_r):
        for j in range(grid_c):
            ZZ[i, j] = pred_fun(XX[i, j], YY[i, j])
    pl.contourf(XX, YY, ZZ, 100, cmap = pl.cm.coolwarm, vmin= 1, vmax=3)
    pl.colorbar()
    pl.xlabel("x")
    pl.ylabel("y")
    
def gen_pred_fun(clf):
    def pred_fun(x1, x2):
        x = np.array([[x1, x2]])
        return clf.predict(x)[0]
    return pred_fun

def list_cm(cm,classes):     #función para generar de una forma más visual la matriz de confusión
    if len(cm)==2:
      cm.astype(int)
      row_0 =['','Valor','Verdadero']
      row_1 =['-',classes[0],classes[1]]
      row_2 =[classes[0],cm[0,0],cm[1,0]]
      row_3 =[classes[1],cm[0,1],cm[1,1]]
      table = zip(row_0,row_1, row_2, row_3)
      headers = ['', '', 'Valor', 'Predicho']  
      return print(tabulate(table, headers=headers, floatfmt=".0f"))
    else:
      cm.astype(int)
      row_0 =['','Valor','Verdadero','']
      row_1 =['-',np.int(classes[0]),classes[1],classes[2]]
      row_2 =[classes[0],cm[0,0],cm[1,0],cm[2,0]]
      row_3 =[classes[1],cm[0,1],cm[1,1],cm[2,1]]
      row_4 =[classes[2],cm[0,2],cm[1,2],cm[2,2]]
      table = zip(row_0,row_1, row_2, row_3, row_4)
      headers = ['', '', 'Valor', 'Predicho', '']  
      return print(tabulate(table, headers=headers, floatfmt=".0f")) 

import itertools

def plot_confusion_matrix(cm, classes,
                          title='Matriz de confusión',
                          cmap=pl.cm.Blues):
    print(cm) # Confusion matrix

    pl.imshow(cm, interpolation='nearest', cmap=cmap) # Pintamos la matriz como una imagen
    pl.title(title)
    pl.colorbar()
    tick_marks = np.arange(len(classes))
    pl.xticks(tick_marks, classes, rotation=45) # Nombre de las clases en X
    pl.yticks(tick_marks, classes) # Nombre de las clases en Y

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        pl.text(j, i, format(cm[i, j], 'd'),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black") 
        # Anotamos cada sección de la imagen con su valor correspondiente en la matriz

    pl.tight_layout()
    pl.ylabel('Valor de verdad')
    pl.xlabel('Valor predicho')

    
    %matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pylab as pl
from sklearn.datasets import make_circles
from matplotlib.colors import Normalize


# Función para visualizar un conjunto de datos en 2D
def plot_data(X, y):
    y_unique = np.unique(y)
    colors = pl.cm.rainbow(np.linspace(0.0, 1.0, y_unique.size))
    for this_y, color in zip(y_unique, colors):
        this_X = X[y == this_y]
        pl.scatter(this_X[:, 0], this_X[:, 1],  c=color,
                    alpha=0.5, edgecolor='k',
                    label="Class %s" % this_y)
    pl.legend(loc="best")
    pl.title("Data")
    
# Función para visualizar de la superficie de decisión de un clasificador
def plot_decision_region(X, pred_fun):
    min_x = np.min(X[:, 0])
    max_x = np.max(X[:, 0])
    min_y = np.min(X[:, 1])
    max_y = np.max(X[:, 1])
    min_x = min_x - (max_x - min_x) * 0.05
    max_x = max_x + (max_x - min_x) * 0.05
    min_y = min_y - (max_y - min_y) * 0.05
    max_y = max_y + (max_y - min_y) * 0.05
    x_vals = np.linspace(min_x, max_x, 100)
    y_vals = np.linspace(min_y, max_y, 100)
    XX, YY = np.meshgrid(x_vals, y_vals)
    grid_r, grid_c = XX.shape
    ZZ = np.zeros((grid_r, grid_c))
    for i in range(grid_r):
        for j in range(grid_c):
            ZZ[i, j] = pred_fun(XX[i, j], YY[i, j])
    pl.contourf(XX, YY, ZZ, 100, cmap = pl.cm.coolwarm, vmin= -1, vmax=2)
    pl.colorbar()
    pl.xlabel("x")
    pl.ylabel("y")
    
class MidpointNormalize(Normalize):

    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y))
    
def gen_pred_fun(clf):
    def pred_fun(x1, x2):
        x = np.array([[x1, x2]])
        return clf.predict(x)[0]
    return pred_fun

def list_cm(cm,classes):     #función para generar de una forma más visual la matriz de confusión
    if len(cm)==2:
      cm.astype(int)
      row_0 =['','Valor','Verdadero']
      row_1 =['-',classes[0],classes[1]]
      row_2 =[classes[0],cm[0,0],cm[1,0]]
      row_3 =[classes[1],cm[0,1],cm[1,1]]
      table = zip(row_0,row_1, row_2, row_3)
      headers = ['', '', 'Valor', 'Predicho']  
      return print(tabulate(table, headers=headers, floatfmt=".0f"))
    else:
      cm.astype(int)
      row_0 =['','Valor','Verdadero','']
      row_1 =['-',np.int(classes[0]),classes[1],classes[2]]
      row_2 =[classes[0],cm[0,0],cm[1,0],cm[2,0]]
      row_3 =[classes[1],cm[0,1],cm[1,1],cm[2,1]]
      row_4 =[classes[2],cm[0,2],cm[1,2],cm[2,2]]
      table = zip(row_0,row_1, row_2, row_3, row_4)
      headers = ['', '', 'Valor', 'Predicho', '']
      return print(tabulate(table, headers=headers, floatfmt=".0f")) 

In [ ]:
import os
os.getcwd()

In [ ]:
import pandas as pd
import numpy as np
import csv
df = pd.read_csv("BD/BaseFinal.csv", encoding = 'latin1', sep = ";", decimal = ",")

In [ ]:
df.head(3)

In [ ]:
df.columns

In [ ]:
categoricas=['genero', 'clasedecontrato', 'regional', 'departamento', 'ciudad', 'tipodevinculo', 'tipodeposicion', 
             'posicion', 'horassemanalespromedio', 'motivodeaprobaciongral', 'turnoposicion', 'familiasdecargo', 
             'divisiondepersonal', 'subdivisiondepersonal', 'areaorganizativa', 'nopersonaljefe','posicioncargodejefe',
             'Categoria', 'Segmento_poblacional','estado_civil', 'nivel_academico', 'segmento_grupo_familiar', 
             'DepartamentoPersona', 'MunicipioPersona','TipoPoblado',  'EstratoPersona', 'filial_famisanar', 
             'filial_ips_colsubsidio', 'filial_ips_cafam', 'filial_otra_ips', 'filial_pac_famisanar', 'filial_bancompartir', 
             'marca_aventurero','marca_tu_club','filial_suramericana', 'filial_pac_suramericana', 'filial_proteccion', 
             'filial_pension_obligatoria','filial_pension_voluntaria', 'filial_cesantias', 'CS_As_Hipo', 'CS_As_Libr', 
             'CS_As_NoLibranza', 'CS_As_Titular', 'CS_CO_CsConv', 'MS_Drog', 'MS_Sup', 'Ryt_AgViaje', 'Ryt_ClBellavista', 
             'Ryt_ClCll195','Ryt_ClColina', 'Ryt_ClCubo', 'Ryt_HoAlcaravan', 'Ryt_HoAthan', 'Ryt_HoPaipa', 'Ryt_HoPeÃ±alisa', 
             'Ryt_Piscilago', 'Ryt_PrDeport', 'Ryt_PrRecRyt', 'Salud_NoPos', 'Salud_Pos', 'Riesgo']

for var in categoricas:
    df[var] = df[var].astype("category")

In [ ]:
df=df.drop(['nodocumento','nombre','fechadeingreso','ubicacion','FechaNacimiento','unidaddirectiva','unidadestrategica','unidadorganizativa','Poblado','PrimeraFalta'], axis=1)

In [ ]:
df.apply(lambda x: x.isnull().sum(), axis=0)

In [ ]:
categoricas=['genero', 'clasedecontrato', 'regional', 'departamento', 'ciudad', 'tipodevinculo', 'tipodeposicion', 
             'posicion', 'horassemanalespromedio', 'motivodeaprobaciongral', 'turnoposicion', 'familiasdecargo', 
             'divisiondepersonal', 'subdivisiondepersonal', 'areaorganizativa', 'nopersonaljefe','posicioncargodejefe',
             'Categoria', 'Segmento_poblacional','estado_civil', 'nivel_academico', 'segmento_grupo_familiar', 
             'DepartamentoPersona', 'MunicipioPersona','TipoPoblado',  'EstratoPersona', 'filial_famisanar', 
             'filial_ips_colsubsidio', 'filial_ips_cafam', 'filial_otra_ips', 'filial_pac_famisanar', 'filial_bancompartir', 
             'marca_aventurero','marca_tu_club','filial_suramericana', 'filial_pac_suramericana', 'filial_proteccion', 
             'filial_pension_obligatoria','filial_pension_voluntaria', 'filial_cesantias', 'CS_As_Hipo', 'CS_As_Libr', 
             'CS_As_NoLibranza', 'CS_As_Titular', 'CS_CO_CsConv', 'MS_Drog', 'MS_Sup', 'Ryt_AgViaje', 'Ryt_ClBellavista', 
             'Ryt_ClCll195','Ryt_ClColina', 'Ryt_ClCubo', 'Ryt_HoAlcaravan', 'Ryt_HoAthan', 'Ryt_HoPaipa', 'Ryt_HoPeÃ±alisa', 
             'Ryt_Piscilago', 'Ryt_PrDeport', 'Ryt_PrRecRyt', 'Salud_NoPos', 'Salud_Pos']


In [ ]:
df1 = pd.get_dummies(df, columns=categoricas, prefix=['genero', 'clasedecontrato', 'regional', 'departamento', 'ciudad', 'tipodevinculo', 'tipodeposicion', 
             'posicion', 'horassemanalespromedio', 'motivodeaprobaciongral', 'turnoposicion', 'familiasdecargo', 
             'divisiondepersonal', 'subdivisiondepersonal', 'areaorganizativa', 'nopersonaljefe','posicioncargodejefe',
             'Categoria', 'Segmento_poblacional','estado_civil', 'nivel_academico', 'segmento_grupo_familiar', 
             'DepartamentoPersona', 'MunicipioPersona','TipoPoblado',  'EstratoPersona', 'filial_famisanar', 
             'filial_ips_colsubsidio', 'filial_ips_cafam', 'filial_otra_ips', 'filial_pac_famisanar', 'filial_bancompartir', 
             'marca_aventurero','marca_tu_club','filial_suramericana', 'filial_pac_suramericana', 'filial_proteccion', 
             'filial_pension_obligatoria','filial_pension_voluntaria', 'filial_cesantias', 'CS_As_Hipo', 'CS_As_Libr', 
             'CS_As_NoLibranza', 'CS_As_Titular', 'CS_CO_CsConv', 'MS_Drog', 'MS_Sup', 'Ryt_AgViaje', 'Ryt_ClBellavista', 
             'Ryt_ClCll195','Ryt_ClColina', 'Ryt_ClCubo', 'Ryt_HoAlcaravan', 'Ryt_HoAthan', 'Ryt_HoPaipa', 'Ryt_HoPeÃ±alisa', 
             'Ryt_Piscilago', 'Ryt_PrDeport', 'Ryt_PrRecRyt', 'Salud_NoPos', 'Salud_Pos'])

X = df1.drop(['Riesgo'], axis = 1)
y = df[['Riesgo']]

## Train & Test

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=31415)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
sm = SMOTE(ratio=0.5, random_state=31415)
X_res, y_res = sm.fit_sample(X_train, y_train)

### Estimación de Modelos

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.svm import LinearSVC 
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
models = {'GradientBoosting': GradientBoostingClassifier(random_state=31415, n_estimators=1250)
          ,'RandomForest' : RandomForestClassifier(n_jobs=-1)
          ,'DesicionTree' : DecisionTreeClassifier(max_depth=5)
}

for model in models.keys():
    models[model].fit(X_res, y_res)

In [ ]:
y_pred = pd.DataFrame(index=X_test.index, columns=models.keys())
for model in models.keys():
    y_pred[model] = models[model].predict(X_test)

In [ ]:
classes = df['Riesgo'].unique()
print(classes)
for model in models.keys():
    cnf_matrix_1 = confusion_matrix(y_test, y_pred[model])
    print(model)
    list_cm(cnf_matrix_1, classes = classes)

In [ ]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

for model in models.keys():
    print(model)
    print('Accurracy: {}'.format(models[model].score(X_test, y_test)))
    print('Error de clasificación: {}'.format(1 - models[model].score(X_test, y_test)))
    print('Precisión macro: {}'.format(precision_score(y_test, y_pred[model], average='macro')))
    print('Recall macro: {}'.format(recall_score(y_test, y_pred[model], average='macro')))
    print('F1 macro: {}'.format(f1_score(y_test,y_pred[model], average='macro')))
    print('AUC: {}'.format(roc_auc_score(y_test, models[model].predict_proba(X_test)[:, 1]), average='macro'))

## Cross Validation

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

FinalMod=models['GradientBoosting']
FinalMod1=models['DesicionTree']

n_cv=5
scoring = ['accuracy', 'f1_macro', 'precision_macro', 'recall_macro', 'roc_auc']

score=cross_validate(FinalMod, X_res, y_res, cv=n_cv, scoring=scoring)
print("train: \n", score1, "\n")

score=cross_validate(FinalMod, X_test, y_test, cv=n_cv, scoring=scoring)
print("test: \n", score1)

score1=cross_validate(FinalMod1, X_res, y_res, cv=n_cv, scoring=scoring)
print("train: \n", score1, "\n")

score1=cross_validate(FinalMod1, X_test, y_test, cv=n_cv, scoring=scoring)
print("test: \n", score1)

In [ ]:
Proba = FinalMod.predict_proba(X_test)

In [ ]:
Data_Fin = pd.concat([df.reset_index(drop=True), pd.DataFrame(Proba)], axis=1)
Data_Fin.to_csv('Resultados/Riesgo_Calif.csv', sep='|', encoding='utf-8', index=False)

### Feature Importance

In [ ]:
# Almacenamiento de la importancia de los predictores.
#importances = FinalMod.feature_importances_
#indices = np.argsort(importances)

# Gráfico de importancia de los predictores.
#plt.figure(1)
#plt.title('Importancia de los Predictores')
#plt.barh(range(len(indices)), importances[indices], color='b', align='center')
#plt.yticks(range(len(indices)), [X[i] for i in indices])
#plt.xlabel('Importancia Relativa')
from sklearn.externals import joblib
joblib.dump(FinalMod, 'Resultados/GradienteBoosting.pkl')